In [1]:
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os
from dataset import DatasetFromPandas
import models.dcgan as dcgan
import models.mlp as mlp

parser = argparse.ArgumentParser()
parser.add_argument('--pos_data', default='imgs/test.csv', help='path to dataset')
parser.add_argument('--neg_data', default='imgs/test.csv', help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=2, help='input batch size')
parser.add_argument('--nSize', type=int, default=147, help='noise size')
parser.add_argument('--nz', type=int, default=147, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lrD', type=float, default=0.00005, help='learning rate for Critic, default=0.00005')
parser.add_argument('--lrG', type=float, default=0.00005, help='learning rate for Generator, default=0.00005')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda'  , action='store_true', help='enables cuda')
parser.add_argument('--ngpu'  , type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--netP', default='samples/netD_epoch_24.pth', help="path to netP (to continue training)")
parser.add_argument('--clamp_lower', type=float, default=-0.01)
parser.add_argument('--clamp_upper', type=float, default=0.01)
parser.add_argument('--Diters', type=int, default=5, help='number of D iters per each G iter')
parser.add_argument('--noBN', action='store_true', help='use batchnorm or not (only for DCGAN)')
parser.add_argument('--mlp_G', action='store_true',default=True, help='use MLP for G')
parser.add_argument('--mlp_D', action='store_true', default=True,help='use MLP for D')
parser.add_argument('--n_extra_layers', type=int, default=0, help='Number of extra layers on gen and disc')
parser.add_argument('--experiment', default=None, help='Where to store samples and models')
parser.add_argument('--adam', action='store_true', help='Whether to use adam (default is rmsprop)')
opt, unknown = parser.parse_known_args()
print(opt)


if opt.experiment is None:
    opt.experiment = 'samples'
os.system('mkdir {0}'.format(opt.experiment))

opt.manualSeed = random.randint(1, 10000) # fix seed
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

cudnn.benchmark = True

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")




Namespace(Diters=5, adam=False, batchSize=2, beta1=0.5, clamp_lower=-0.01, clamp_upper=0.01, cuda=False, experiment=None, lrD=5e-05, lrG=5e-05, mlp_D=True, mlp_G=True, nSize=147, n_extra_layers=0, ndf=64, neg_data='imgs/test.csv', netD='', netG='', netP='samples/netD_epoch_24.pth', ngf=64, ngpu=1, niter=25, noBN=False, nz=147, pos_data='imgs/test.csv', workers=2)
Random Seed:  8753


In [2]:
pos_data = DatasetFromPandas(opt.pos_data)

neg_data = DatasetFromPandas(opt.neg_data)

dataloader = torch.utils.data.DataLoader(pos_data, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))
neg_dataloader = torch.utils.data.DataLoader(neg_data, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))

ngpu = int(opt.ngpu)
nz = int(opt.nz)
ngf = int(opt.ngf)
ndf = int(opt.ndf)
n_extra_layers = int(opt.n_extra_layers)

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

if opt.noBN:
    netG = dcgan.DCGAN_G_nobn(opt.imageSize, nz, nc, ngf, ngpu, n_extra_layers)
elif opt.mlp_G:
    netG = mlp.MLP_G(nz, nz,  ngf, ngpu)
else:
    netG = dcgan.DCGAN_G(opt.imageSize, nz, nc, ngf, ngpu, n_extra_layers)

netG.apply(weights_init)
if opt.netG != '': # load checkpoint if needed
    netG.load_state_dict(torch.load(opt.netG))
print(netG)

if opt.mlp_D:
    netD = mlp.MLP_D(opt.nSize,  ndf, ngpu)
else:
    netD = dcgan.DCGAN_D(opt.imageSize, nz, nc, ndf, ngpu, n_extra_layers)
    netD.apply(weights_init)

if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))
print(netD)

# if opt.netP != '': # load checkpoint if needed
#     netP.load_state_dict(torch.load(opt.netG))

input = torch.FloatTensor(opt.batchSize, opt.nSize)
noise = torch.FloatTensor(opt.batchSize, nz, 1, 1)
# fixed_noise = torch.FloatTensor(opt.batchSize, nz, 1, 1).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

if opt.cuda:
    netD.cuda()
    netG.cuda()
    input = input.cuda()
    one, mone = one.cuda(), mone.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

# setup optimizer
if opt.adam:
    optimizerD = optim.Adam(netD.parameters(), lr=opt.lrD, betas=(opt.beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=opt.lrG, betas=(opt.beta1, 0.999))
else:
    optimizerD = optim.RMSprop(netD.parameters(), lr = opt.lrD)
    optimizerG = optim.RMSprop(netG.parameters(), lr = opt.lrG)



MLP_G (
  (main): Sequential (
    (0): Linear (147 -> 64)
    (1): ReLU (inplace)
    (2): Linear (64 -> 64)
    (3): ReLU (inplace)
    (4): Linear (64 -> 64)
    (5): ReLU (inplace)
    (6): Linear (64 -> 147)
  )
)
MLP_D (
  (main): Sequential (
    (0): Linear (147 -> 64)
    (1): ReLU (inplace)
    (2): Linear (64 -> 64)
    (3): ReLU (inplace)
    (4): Linear (64 -> 64)
    (5): ReLU (inplace)
    (6): Linear (64 -> 1)
  )
)


NameError: name 'netP' is not defined

In [66]:
len(dataloader)

5

In [78]:
gen_iterations = 0
for epoch in range(opt.niter):
    data_iter = iter(dataloader)
    neg_iter = iter(neg_dataloader)

    i = 0
    while i < len(dataloader):
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update

        # train the discriminator Diters times
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 100
        else:
            Diters = opt.Diters
        j = 0
        while j < Diters and i < len(dataloader):
            j += 1
            # clamp parameters to a cube
            for p in netD.parameters():
                p.data.clamp_(opt.clamp_lower, opt.clamp_upper)

            data = data_iter.next()
            i += 1

            # train with real
            real_cpu = data
            netD.zero_grad()
            # batch_size = real_cpu.size(0)

            if opt.cuda:
                real_cpu = real_cpu.cuda()
            input.resize_as_(real_cpu).copy_(real_cpu)
            inputv = Variable(input)

            errD_real = netD(inputv)
            errD_real.backward(one)

            # train with fake
            try:
                noise = neg_iter.next()
#                 print (noise.size())
            except:
                neg_iter = iter(neg_dataloader)
                noise = neg_iter.next()

            # noise.resize_(opt.batchSize, nz, 1, 1).normal_(0, 1)
            noisev = Variable(noise, volatile = True) # totally freeze netG
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD(inputv)
            errD_fake.backward(mone)
            errD = errD_real - errD_fake
            optimizerD.step()

        ############################
        # (2) Update G network
        ###########################
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        netG.zero_grad()
        # in case our last batch was the tail batch of the dataloader,
        # make sure we feed a full batch of noise
        # noise.resize_(opt.batchSize, nz, 1, 1).normal_(0, 1)
        try:
                noise = neg_iter.next()
                
        except:
                neg_iter = iter(neg_dataloader)
                noise = neg_iter.next()

        noisev = Variable(noise)
        fake = netG(noisev)
        errG = netD(fake)

        errG.backward(one)
        optimizerG.step()
        gen_iterations += 1

        print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, opt.niter, i, len(dataloader), gen_iterations,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
        #print ('err',errD.size(), errG.size(), errD_real.size(), errD_fake.size(),)

        if gen_iterations % 500 == 0:
            # real_cpu = real_cpu.mul(0.5).add(0.5)
            # fake = netG(Variable(fixed_noise, volatile=True))
            # fake.data = fake.data.mul(0.5).add(0.5)
            pass
torch.save(netD.state_dict(), '{0}/netD_epoch_{1}.pth'.format(opt.experiment, epoch))


input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([1, 147])
[0/25][5/5][1] Loss_D: -0.000018 Loss_G: -0.011004 Loss_D_real: -0.011018 Loss_D_fake -0.010999
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([1, 147])
[1/25][5/5][2] Loss_D: -0.000053 Loss_G: -0.010980 Loss_D_real: -0.011018 Loss_D_fake -0.010965
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([1, 147])
[2/25][5/5][3] Loss_D: -0.000096 Loss_G: -0.010953 Loss_D_real: -0.011037 Loss_D_fake -0.010941
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([1, 147])
[3/25][5/5][4] Loss_D: -0.000130 Loss_G: -0.010926 Loss_D_real: -0.011049 Loss_D_fake -0.010919
input torch.Size([2, 147])
input torch.Size([2, 147])
input torch.Size([2, 1

In [59]:
from dataset import DatasetFromPandas


In [6]:
test = DatasetFromPandas('imgs/test.csv')

In [7]:
dataloader = torch.utils.data.DataLoader(test, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))


In [8]:
dataiter = iter(dataloader)

In [10]:
real_cpu = dataiter.next()
inputv = Variable(real_cpu)

In [47]:
test1 = iter(test)

In [51]:
for i in range(20):
    
    try:
#         print (i,'pos')
        print (test1.next().size())
    except:
#         print (i,'neg')
        test = DatasetFromPandas('imgs/test.csv')
        test1 = iter(test)
        print (test1.next().size())

        continue

torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])
torch.Size([147])


In [4]:
netP = mlp.MLP_P(opt.nSize,  ndf, ngpu)

netP.load_state_dict(torch.load(opt.netP))


In [15]:
netP(inputv).data.numpy()

array([[-0.01307047],
       [-0.01302315]], dtype=float32)